In [11]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
from torchvision import datasets, models, transforms
import os
from PIL import Image

In [12]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

transform=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])
class food11:
    def __init__(self,path,transform):
        self.path=path
        self.n_file=sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith('.jpg')])
        print(self.n_file[0].replace('\\','/'))
        self.sample=len(self.n_file)
        self.transform=transform
    def __getitem__(self,index):
        img=Image.open(self.n_file[index].replace('\\','/'))
        img=self.transform(img);
        label=self.n_file[index].replace('\\','/').split("/")[-1].split("_")[0]
        label=label
        label=int(label)
        return img,label
    def __len__(self):
        return self.sample
    

In [13]:
path='./food11/training'
data=food11(path,transform)
dataloader=torch.utils.data.DataLoader(dataset=data,batch_size=128,shuffle=True)
x,_=iter(dataloader).next()

# print(a)

./food11/training/0_0.jpg


In [14]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)

        # input image size: [3, 128, 128]
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
#             nn.Dropout2d(p=0.2),
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
            
#             nn.Dropout2d(p=0.2),
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(4, 4, 0),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(256 * 8 * 8, 256),
#             nn.Dropout(p=0.2),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 11)
        )

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)

        # The extracted feature map must be flatten before going to fully-connected layers.
        x = x.flatten(1)

        # The features are transformed by fully-connected layers to obtain the final logits.
        x = self.fc_layers(x)
        return x
        

In [15]:
epochs=10
lr=0.0003
n_step=len(dataloader)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [16]:
model=Classifier().to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=lr,weight_decay=1e-5)

In [17]:
for epoch in range(epochs):
    for i,(x,y) in enumerate(dataloader):
        x,y = x.to(device), y.to(device)
        pred = model(x)
        loss = criterion(pred,y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f'epoch= {epoch}, step= {i}/{n_step}, loss= {loss:.3f}')

epoch= 0, step= 0/78, loss= 2.397
epoch= 0, step= 10/78, loss= 2.183
epoch= 0, step= 20/78, loss= 1.951
epoch= 0, step= 30/78, loss= 1.861
epoch= 0, step= 40/78, loss= 1.830
epoch= 0, step= 50/78, loss= 1.958
epoch= 0, step= 60/78, loss= 1.872
epoch= 0, step= 70/78, loss= 1.701
epoch= 1, step= 0/78, loss= 1.593
epoch= 1, step= 10/78, loss= 1.673
epoch= 1, step= 20/78, loss= 1.619
epoch= 1, step= 30/78, loss= 1.665
epoch= 1, step= 40/78, loss= 1.903
epoch= 1, step= 50/78, loss= 1.387
epoch= 1, step= 60/78, loss= 1.403
epoch= 1, step= 70/78, loss= 1.553
epoch= 2, step= 0/78, loss= 1.437
epoch= 2, step= 10/78, loss= 1.466
epoch= 2, step= 20/78, loss= 1.487
epoch= 2, step= 30/78, loss= 1.266
epoch= 2, step= 40/78, loss= 1.306
epoch= 2, step= 50/78, loss= 1.365
epoch= 2, step= 60/78, loss= 1.391
epoch= 2, step= 70/78, loss= 1.504
epoch= 3, step= 0/78, loss= 1.441
epoch= 3, step= 10/78, loss= 1.238
epoch= 3, step= 20/78, loss= 1.282
epoch= 3, step= 30/78, loss= 1.373
epoch= 3, step= 40/78, l